In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
!curl -ssL https://magic.modular.com/ | bash

In [ ]:
import os
os.environ['PATH'] += ':/root/.modular/bin'

In [ ]:
!magic init mojo_kernels --format mojoproject

In [ ]:
%cd mojo_kernels/

In [36]:
%%writefile tensor_indexer_kernel.mojo

from gpu import block_idx, thread_idx
from gpu.host import DeviceContext
from math import iota
from layout import Layout, LayoutTensor

alias dtype = DType.uint32
alias blocks = 4
alias threads = 4
var elems_count = blocks * threads
alias layout = Layout.row_major(blocks, threads)
alias InputTensor = LayoutTensor[dtype, layout, MutableAnyOrigin]

fn tensor_indexer_kernel(input_tensor: InputTensor):
    input_tensor[0, 0] = 200
    var bid = block_idx.x
    var tid = thread_idx.x
    print("Block id: ", bid, ", thread id: ", tid, ", value: ", input_tensor[bid, tid])
    #print("Value: ", input_tensor[bid, tid])

def main():
    var ctx = DeviceContext()
    var device_input_buff = ctx.enqueue_create_buffer[dtype](elems_count)

    with device_input_buff.map_to_host() as host_input_buff:
      iota(host_input_buff.unsafe_ptr(), elems_count)
      print(host_input_buff)

    #var input_tensor = LayoutTensor[dtype, layout](device_input_buff)
    var input_tensor = InputTensor(device_input_buff)
    ctx.enqueue_function[tensor_indexer_kernel](input_tensor, grid_dim=blocks, block_dim=threads)
    ctx.synchronize()



Overwriting tensor_indexer_kernel.mojo


In [40]:
!magic run mojo tensor_indexer_kernel.mojo

HostBuffer([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
Block id:  2 , thread id:  0 , value:  8
Block id:  2 , thread id:  1 , value:  9
Block id:  2 , thread id:  2 , value:  10
Block id:  2 , thread id:  3 , value:  11
Block id:  0 , thread id:  0 , value:  200
Block id:  0 , thread id:  1 , value:  1
Block id:  0 , thread id:  2 , value:  2
Block id:  0 , thread id:  3 , value:  3
Block id:  1 , thread id:  0 , value:  4
Block id:  1 , thread id:  1 , value:  5
Block id:  1 , thread id:  2 , value:  6
Block id:  1 , thread id:  3 , value:  7
Block id:  3 , thread id:  0 , value:  12
Block id:  3 , thread id:  1 , value:  13
Block id:  3 , thread id:  2 , value:  14
Block id:  3 , thread id:  3 , value:  15
